 Nom Etudiant : Seck

Prenom Etudiant: Abdou Samath

Classe :ING3 IPSL

In [1]:
import org.apache.spark.sql.SparkSession
val spark = SparkSession.builder.
                        appName("Projet Spark").
                        config("spark.ui.port", "0").
                        master("local[*]").
                        getOrCreate()

Intitializing Scala interpreter ...

Spark Web UI available at http://10.151.52.142:4040
SparkContext available as 'sc' (version = 3.3.1, master = local[*], app id = local-1685201078622)
SparkSession available as 'spark'


23/05/27 15:24:51 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


import org.apache.spark.sql.SparkSession
spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@77283bfa


In [2]:
val eventsPath = "datasets/events/events.json"

eventsPath: String = datasets/events/events.json


In [3]:
val eventsDF = spark.read.json(eventsPath)
eventsDF.show()

+-------+------------------+----------+------------------------+----------------+--------------------+--------------------+--------------+--------------------------+-----------------+
| device|         ecommerce|event_name|event_previous_timestamp| event_timestamp|                 geo|               items|traffic_source|user_first_touch_timestamp|          user_id|
+-------+------------------+----------+------------------------+----------------+--------------------+--------------------+--------------+--------------------------+-----------------+
|  macOS|{null, null, null}|  warranty|        1593878899217692|1593878946592107|      {Montrose, MI}|                  []|        google|          1593878899217692|UA000000107379500|
|Windows|{null, null, null}|     press|        1593876662175340|1593877011756535|   {Northampton, MA}|                  []|        google|          1593876662175340|UA000000107359357|
|  macOS|{null, null, null}|  add_item|        1593878792892652|1593878815459100

eventsDF: org.apache.spark.sql.DataFrame = [device: string, ecommerce: struct<purchase_revenue_in_usd: double, total_item_quantity: bigint ... 1 more field> ... 8 more fields]


## 1. Extraire les revenus d'achat pour chaque événement

In [4]:
import org.apache.spark.sql.functions._

val revenueDF = eventsDF.withColumn("revenue", col("ecommerce.purchase_revenue_in_usd"))

revenueDF.show()


+-------+------------------+----------+------------------------+----------------+--------------------+--------------------+--------------+--------------------------+-----------------+-------+
| device|         ecommerce|event_name|event_previous_timestamp| event_timestamp|                 geo|               items|traffic_source|user_first_touch_timestamp|          user_id|revenue|
+-------+------------------+----------+------------------------+----------------+--------------------+--------------------+--------------+--------------------------+-----------------+-------+
|  macOS|{null, null, null}|  warranty|        1593878899217692|1593878946592107|      {Montrose, MI}|                  []|        google|          1593878899217692|UA000000107379500|   null|
|Windows|{null, null, null}|     press|        1593876662175340|1593877011756535|   {Northampton, MA}|                  []|        google|          1593876662175340|UA000000107359357|   null|
|  macOS|{null, null, null}|  add_item| 

import org.apache.spark.sql.functions._
revenueDF: org.apache.spark.sql.DataFrame = [device: string, ecommerce: struct<purchase_revenue_in_usd: double, total_item_quantity: bigint ... 1 more field> ... 9 more fields]


## 2. Filtrons les événements dont le revenu n'est pas null

In [5]:
val purchasesDF = revenueDF.filter(col("revenue").isNotNull)

purchasesDF.show()


+---------+--------------+----------+------------------------+----------------+--------------------+--------------------+--------------+--------------------------+-----------------+-------+
|   device|     ecommerce|event_name|event_previous_timestamp| event_timestamp|                 geo|               items|traffic_source|user_first_touch_timestamp|          user_id|revenue|
+---------+--------------+----------+------------------------+----------------+--------------------+--------------------+--------------+--------------------------+-----------------+-------+
|    Linux|{1195.0, 1, 1}|  finalize|        1593878893766134|1593878897648871|       {Shawnee, KS}|[{null, M_STAN_K,...|        google|          1593876996316576|UA000000107362263| 1195.0|
|      iOS|{1045.0, 1, 1}|  finalize|        1593878485345763|1593878487460247|       {Detroit, MI}|[{null, M_STAN_Q,...|      facebook|          1593877230282722|UA000000107364432| 1045.0|
|  Android| {595.0, 1, 1}|  finalize|        15938

purchasesDF: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [device: string, ecommerce: struct<purchase_revenue_in_usd: double, total_item_quantity: bigint ... 1 more field> ... 9 more fields]


## 3. Vérifions quels sont les types d'événements qui génèrent des revenus



In [6]:
val distinctDF = purchasesDF.select("event_name").distinct()

distinctDF.show()


+----------+
|event_name|
+----------+
|  finalize|
+----------+



distinctDF: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [event_name: string]


# ## 4. Supprimons la colonne inutile
Puisqu'il n'y a qu'un seul type d'événement, supprimez event_name de purchasesDF.

In [7]:
val cleanDF = purchasesDF.drop("event_name")

cleanDF.show()


+---------+--------------+------------------------+----------------+--------------------+--------------------+--------------+--------------------------+-----------------+-------+
|   device|     ecommerce|event_previous_timestamp| event_timestamp|                 geo|               items|traffic_source|user_first_touch_timestamp|          user_id|revenue|
+---------+--------------+------------------------+----------------+--------------------+--------------------+--------------+--------------------------+-----------------+-------+
|    Linux|{1195.0, 1, 1}|        1593878893766134|1593878897648871|       {Shawnee, KS}|[{null, M_STAN_K,...|        google|          1593876996316576|UA000000107362263| 1195.0|
|      iOS|{1045.0, 1, 1}|        1593878485345763|1593878487460247|       {Detroit, MI}|[{null, M_STAN_Q,...|      facebook|          1593877230282722|UA000000107364432| 1045.0|
|  Android| {595.0, 1, 1}|        1593877930076602|1593878966392505|  {East Chicago, IN}|[{null, M_STAN_T

cleanDF: org.apache.spark.sql.DataFrame = [device: string, ecommerce: struct<purchase_revenue_in_usd: double, total_item_quantity: bigint ... 1 more field> ... 8 more fields]


## 5. Revenus cumulés par source de trafic
Obtenir la somme de revenue comme total_rev
Obtenir la moyenne de revenue comme avg_rev

In [8]:
import org.apache.spark.sql.functions._

val trafficDF = cleanDF.groupBy("traffic_source")
  .agg(sum("revenue").alias("total_rev"), avg("revenue").alias("avg_rev"))

trafficDF.show


+--------------+---------+-----------------+
|traffic_source|total_rev|          avg_rev|
+--------------+---------+-----------------+
|     instagram|   8160.0|           1020.0|
|        direct|   9129.0|         1141.125|
|       youtube|   3040.0|           1520.0|
|         email|  36935.4|998.2540540540541|
|      facebook|  12952.0|996.3076923076923|
|        google|  28936.0|964.5333333333333|
+--------------+---------+-----------------+



import org.apache.spark.sql.functions._
trafficDF: org.apache.spark.sql.DataFrame = [traffic_source: string, total_rev: double ... 1 more field]


## 6. Obtenir les trois principales sources de trafic par revenu total

In [14]:
import org.apache.spark.sql.functions._

val topTrafficDF = trafficDF
  .groupBy("traffic_source")
  .agg(sum("total_rev").alias("total_rev"), avg("avg_rev").alias("avg_rev"))
  .orderBy(desc("total_rev"))
  .limit(3)

topTrafficDF.show()



+--------------+---------+-----------------+
|traffic_source|total_rev|          avg_rev|
+--------------+---------+-----------------+
|         email|  36935.4|998.2540540540541|
|        google|  28936.0|964.5333333333333|
|      facebook|  12952.0|996.3076923076923|
+--------------+---------+-----------------+



import org.apache.spark.sql.functions._
topTrafficDF: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [traffic_source: string, total_rev: double ... 1 more field]


## 7. Nettoyons les colonnes de revenus pour avoir deux décimales


In [15]:
import org.apache.spark.sql.functions._

val finalDF = topTrafficDF
  .withColumn("avg_rev", round(col("avg_rev"), 2))
  .withColumn("total_rev", round(col("total_rev"), 2))

finalDF.show()


+--------------+---------+-------+
|traffic_source|total_rev|avg_rev|
+--------------+---------+-------+
|         email|  36935.4| 998.25|
|        google|  28936.0| 964.53|
|      facebook|  12952.0| 996.31|
+--------------+---------+-------+



import org.apache.spark.sql.functions._
finalDF: org.apache.spark.sql.DataFrame = [traffic_source: string, total_rev: double ... 1 more field]


## 8. Sauvegardons les données


In [17]:
finalDF.write.parquet("projetBigData/projetBigData.parquet")
